In [37]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math as ma

In [38]:
# initial orbital state 
x=-9141.878
y=-1648.0758
z=4141.679
vx=-1.153
vy=-5.31
vz=-2.898

# final orbital state
a=12940
e=0.2173
i=0.8692
Om=1.448
w=2.721
f=2.827

mu=398600.433

In [39]:
#Convert Keplerian elements to Cartesian.
def kep2cart(kep, GM):
    a,e,i,Om,w,f = kep
    p = a*(1-e*e)
    sqrt_GM = np.sqrt(GM/p)
    
    sini, cosi = np.sin(i), np.cos(i)
    sinOm, cosOm = np.sin(Om), np.cos(Om)
    sinw, cosw = np.sin(w), np. cos(w)
    sinf, cosf = np.sin(f), np.cos(f)
    
    rx = p*cosf/(1+e*cosf)
    ry = p*sinf/(1+e*cosf)
    v_x = -sqrt_GM*sinf
    v_y = sqrt_GM*(e+cosf)
    
    R11 = cosOm*cosw - sinOm*cosi*sinw
    R12 = -cosOm*sinw - cosw*cosi*sinOm
    R13 = sini*sinOm
    
    R21 = sinOm*cosw + cosOm*cosi*sinw
    R22 = -sinOm*sinw + cosOm*cosi*cosw
    R23 = -sini*cosOm
    
    R31 = sini*sinw
    R32 = sini*cosw
    R33 = cosi
    
    x = float(R11*rx + R12*ry)
    y = float(R21*rx + R22*ry)
    z = float(R31*rx + R32*ry)
    vx = float(R11*v_x + R12*v_y)
    vy = float(R21*v_x + R22*v_y)
    vz = float(R31*v_x + R32*v_y)
    
    return [x,y,z,vx,vy,vz]

In [40]:
#Convert Cartesian elements to Keplerian.
def cart2kep(cart, GM):
    x,y,z,vx,vy,vz = cart
    eps = 1e-14
    r = np.sqrt(x*x+y*y+z*z)
    rx = x
    ry = y 
    rz = z
    v = np.sqrt(vx*vx+vy*vy+vz*vz)
    E = (1/2)*v*v  - (mu/r)
    a = -mu/(2*E)
    hx = y*vz - z*vy
    hy = -x*vz + z*vx
    hz = x*vy - y*vx
    h = np.sqrt(hx*hx+hy*hy+hz*hz)
    i = np.arccos(hz/h)
    ex = (1/mu)*((v*v-mu/r)*rx - (rx*vx + ry*vy + rz*vz)*vx)
    ey = (1/mu)*((v*v-mu/r)*ry - (rx*vx + ry*vy + rz*vz)*vy)
    ez = (1/mu)*((v*v-mu/r)*rz - (rx*vx + ry*vy + rz*vz)*vz)
    e = np.sqrt(ex*ex+ey*ey+ez*ez)
    nx = -hy
    ny = hx
    nz = 0
    n = np.sqrt(nx*nx+ny*ny+nz*nz)
    
    if ny>=0:
        Om = np.arccos(nx/n)
        
    if ny<0:
        Om = 2*np.pi - np.arccos(nx/n)
    
    if e>eps:
        if n>eps:
             w = np.arccos((nx*ex+ny*ey)/n*e)
        if ez<0:
             w = 2*np.pi - np.arccos((nx*ex+ny*ey)/n*e)
   
    v_r = (x*vx+y*vy+z*vz)/r
    
    
    if v_r>=0:
        f = np.arccos((ex*rx+ey*ry+ez*rz)/(e*r))
                      
    if v_r<0:
        f = 2*np.pi - np.arccos((ex*rx+ey*ry+ez*rz)/(e*r))        
     
    
    return [a,e,i,Om,w,f]



In [41]:
IC = [ a , e, np.deg2rad(i), np.deg2rad(Om), np.deg2rad(w), f ];
ICCAR = kep2cart(IC,mu);

cart1=[x,y,z,vx,vy,vz]
C=cart2kep(cart1,mu)
a1=C[0]
e1=C[1]
w1=C[4]
f1=C[5]
Om1=C[3]
i1=C[2]
dOm=Om-Om1
di=i-i1
print("Om=",Om1)
print("a=",a1)
print("e=",e1)
print("i=",i1)
print("f=",f1)
print("w=",w1)
print("ΔΩ=",dOm)
print("Δi=",di)

T_per=2*np.pi*(np.sqrt(a1**3/mu))

Om= 0.7080148312799239
a= 9852.198421980704
e= 0.12072807884964891
i= 0.7230491866394563
f= 1.9566544240267003
w= 1.5581618269475754
ΔΩ= 0.739985168720076
Δi= 0.1461508133605437


In [42]:
def changeplane(ai,ei,i_i,Omi,wi,i_f,Omf):
    dOm=Omf-Omi
    a=ma.acos(np.cos(i_i)*np.cos(i_f)+np.sin(i_i)*np.sin(i_f)*np.cos(dOm))

    cosu2=(np.cos(i_i)-(np.cos(a))*(np.cos(i_f)))/(np.sin(a)*np.sin(i_f))
    cosu1=(np.cos(i_f)-(np.cos(a))*(np.cos(i_i)))/(np.sin(a)*np.sin(i_i))

    sinu1=(np.sin(i_i)*(np.sin(dOm)/np.sin(a)))
    sinu2=(np.sin(i_f)*(np.sin(dOm)/np.sin(a)))
    u1=np.arctan2(sinu1,cosu1)
    u2=np.arctan2(sinu2,cosu2)
    thetaf=u1-w1
    wf=u2-u1+w1
    p=ai*(1-ei**2)
    Dv=(np.sqrt(mu/p))*(1+np.cos(thetaf))*(np.sin(a/2))
    
    return [Dv,wf,thetaf]
    

In [43]:
def changepericenter(ai,ei,wi,thetai,wt):
    dw=wt-wi
    theta2a=dw/2
    theta3a=2*np.pi-theta2a
    p=ai*(1-ei**2)
    Dv=2*(np.sqrt(mu/p))*(np.sin(dw/2))
    wf=wt
    
    return [Dv,wf,theta2a,theta3a]

In [50]:
def timeofflight(a,e,theta1,theta2,ti):
    
    E=np.arctan(np.sqrt((1-e)/(1+e))*np.tan(theta2/2))
    M=E-e*np.sin(E)
    n=np.sqrt(mu/a)
    t=M/n
    
    if theta2>theta1:
        Dt=t-ti
    else:
        Dt=t-ti+T_per
        
    return [t,Dt]    

In [51]:
def hohmann(ai,ei,af,ef):
    atrans=(ai*(1-ei)+af*(1+ef))/2
    etrans=(af*(1+ef)-ai*(1-ei))/(ai*(1-ei)+af*(1+ef))
    tof=np.pi*(np.sqrt((atrans**3)/mu))
    uc1=np.sqrt(mu/(ai*(1-ei)))
    uc2=np.sqrt(mu/(af*(1+ef)))
    up=np.sqrt((mu/atrans)*((1+etrans)/(1-etrans)))
    ua=np.sqrt((mu/atrans)*((1-etrans)/(1+etrans)))
    du1=up-uc1
    du2=uc2-ua
    Dv=du1+du2
    
    return [atrans,etrans,tof,Dv] 

In [52]:
A1=changeplane(a1,e1,i1,Om1,w1,i,Om); # όχι change of plane μόνο υπολογισμός γωνίας 
A2=timeofflight(a1,e1,f1,A1[2],0);

B1=changeplane(a1,e1,i1,Om1,A1[1],i,Om);
B2=changepericenter(a1,e1,A1[1],A1[2],B1[1]);
B3=timeofflight(a1,e1,B1[2],B2[2],A2[0]);

C1=timeofflight(a1,e1,B2[3],0,B3[0]);
C2=hohmann(a1,e1,a,e);


D1=timeofflight(a,e,np.pi,f,C2[2]);


TOF=A2[1]+B3[1]+C2[2]+D1[1]
print("The time-of-flight is:",TOF,"sec")

DV=B1[0]+B2[0]+C2[3]
print("The total ΔV cost is:",DV)

The time-of-flight is: 29196.788242748593 sec
The total ΔV cost is: 5.139589927745984
